## Ceneo Scraper

## Struktura opinii w serwisie Ceneo.pl

|składowa|selektor|zmienna|
|--------|--------|-------|
|identyfikator opinii|["data-entry-id"]|opinion_id|
|autora|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation > em|recommend|
|liczba gwiazdek|span.user-post__score-count|stars|
|treść opinii|div.user-post__text|content|
|data wystawienia opinii|span.user-post__published > time:nth-child(1)["datatime]|opinion_date|
|data zakupu produktu|span.user-post__published > time:nth-child(2)["datatime]|purchase_date|
|listę wad|div.review_feature__title--negatives ~ div.review-feature__item|cons|
|listę zalet|div.review_feature__title--positives ~ div.review-feature__item|pros|
|ile osób uznało opinię za przydatną|button.vote-yes["data-total-vote"]|up_vote|
|ile osób uznało opinię za nieprzydatną|button.vote-no["data-total-vote"]|down_vote|

In [46]:
selectors = {
        "opinion_id" : (None,"data-entry-id"),
        "author" : ("span.user-post__author-name",),
        "recommend" : ("span.user-post__author-recomendation > em",),
        "stars" : ("span.user-post__score-count",),
        "content" : ("div.user-post__text",),
        "cons" : ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
        "pros" : ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
        "opinion_date" : ("span.user-post__published > time:nth-child(1)","datetime"),
        "purchase_date" : ("span.user-post__published > time:nth-child(2)","datetime"),
        "up_vote" : ("button.vote-yes","data-total-vote"),
        "down_vote" :("button.vote-no","data-total-vote"),
}

In [47]:
def extract(ancestor, selector=None, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

1. Import bibliotek

In [48]:
import os
import json
import requests
from bs4 import BeautifulSoup

2. Wczytanie kodu produktu w którym mają zostać pobrane opinie

In [49]:
#product_id= "151210226"
product_id=input("Podaj kod produktu: ")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

3. Pobranie wszystkich opinii o wskazanym produkcie z serwisu Ceneo.pl

In [50]:
all_opinions=[]
while(url):
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl"+extract(page, "a.pagination__next", "href")
    except TypeError:
        url =  None    

In [51]:
all_opinions

[{'opinion_id': '17886609',
  'author': 'w...7',
  'recommend': 'Polecam',
  'stars': '5/5',
  'content': 'Bardzo dobry sprzed polecam sprzęt od firmy TCL',
  'cons': [],
  'pros': ['jakość dźwięku', 'jakość obrazu', 'łatwość obsługi', 'wygląd'],
  'opinion_date': '2023-09-10 11:53:31',
  'purchase_date': '2023-09-01 06:41:04',
  'up_vote': '4',
  'down_vote': '0'},
 {'opinion_id': '17950592',
  'author': 'J...9',
  'recommend': None,
  'stars': '3/5',
  'content': 'Spoiler !   nie kupiłabym już ! \nPlusy\n\nCena\n+ Oprogramowanie\n+ Wygląd TV \n+ Rozmieszczenie wtyczek typu HDMI\n\nMinusy\n- Nie da się skonfigurować TV przez smartfona\n- Brak intuicyjności jeśli chodzi o kalibracje obrazu ( za dużo opcji zmiany )\n- Aplikacje czasami się wieszają przy przełączaniu \n- Słabe głośniki \n- Tandetny plastikowy pilot (jakość plastiku dramat) \n- Długi czas reakcji na wł się TV\n- By wł tv lub przełączyć trzeba stać na wprost TV lub nakierowywać pilot',
  'cons': ['jakość dźwięku', 'łatwość

In [52]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")


4. Zapis opinii o produkcie do pliku JSON

In [56]:
if not os.path.exists('opinions'):
    os.mkdir('opinions')
with open(f'opinions/{product_id}.json', "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
